# NFL Radio Affiliates: Seahawks vs Patriots
### SBE Meeting Demo — What Can You Do With Python + Broadcast Data?

In this notebook we'll explore two CSV files exported from the **Radioland** broadcast database:
- `seahawks_affiliates.csv` — all AM & FM stations carrying Seattle Seahawks games
- `patriots_affiliates.csv` — all AM & FM stations carrying New England Patriots games

We'll walk through **10 steps** — loading data, counting stations, comparing AM vs FM,
mapping affiliate locations, and more. No prior Python experience required!

> **How to run:** Click each code cell and press **Shift + Enter** (or the ▶ button) to execute it.

---
## Step 1 — Upload the CSV Files
Run this cell and use the **Choose Files** button to upload both CSVs from your computer.

In [ ]:
from google.colab import files

print("Upload seahawks_affiliates.csv and patriots_affiliates.csv")
uploaded = files.upload()

---
## Step 2 — Load the Data into Pandas
**Pandas** is the go-to Python library for working with tabular data (think of it as
Excel on steroids). We load each CSV into a *DataFrame* — basically a spreadsheet in memory.

In [ ]:
import pandas as pd

seahawks = pd.read_csv('seahawks_affiliates.csv')
patriots = pd.read_csv('patriots_affiliates.csv')

print(f"Seahawks affiliates: {len(seahawks)} stations")
print(f"Patriots affiliates: {len(patriots)} stations")
print()

# Preview the first few rows of the Seahawks data
seahawks.head()

---
## Step 3 — Which Team Has More Affiliates?
A simple bar chart tells the story at a glance. **Matplotlib** is Python's
core plotting library — virtually every chart you see in data science starts here.

In [ ]:
import matplotlib.pyplot as plt

teams = ['Seahawks', 'Patriots']
counts = [len(seahawks), len(patriots)]
colors = ['#002244', '#002244']  # Both teams use navy — we'll accent differently

fig, ax = plt.subplots(figsize=(6, 4))
bars = ax.bar(teams, counts, color=['#69BE28', '#C60C30'], edgecolor='black', width=0.5)
ax.set_ylabel('Number of Stations')
ax.set_title('Total Radio Affiliates: Seahawks vs Patriots')

# Put the count on top of each bar
for bar, count in zip(bars, counts):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
            str(count), ha='center', va='bottom', fontweight='bold', fontsize=14)

plt.tight_layout()
plt.show()

---
## Step 4 — AM vs FM Breakdown
How does each team split between the AM and FM bands? This is a *grouped* bar chart —
we break each team's total into AM and FM counts side by side.

In [ ]:
import numpy as np

sea_am = len(seahawks[seahawks['band'] == 'AM'])
sea_fm = len(seahawks[seahawks['band'] == 'FM'])
pat_am = len(patriots[patriots['band'] == 'AM'])
pat_fm = len(patriots[patriots['band'] == 'FM'])

x = np.arange(2)
width = 0.3

fig, ax = plt.subplots(figsize=(7, 4))
am_bars = ax.bar(x - width/2, [sea_am, pat_am], width, label='AM', color='#FFD700', edgecolor='black')
fm_bars = ax.bar(x + width/2, [sea_fm, pat_fm], width, label='FM', color='#1E90FF', edgecolor='black')

ax.set_xticks(x)
ax.set_xticklabels(['Seahawks', 'Patriots'])
ax.set_ylabel('Number of Stations')
ax.set_title('AM vs FM Affiliates by Team')
ax.legend()

# Label each bar
for bars in [am_bars, fm_bars]:
    for bar in bars:
        h = bar.get_height()
        if h > 0:
            ax.text(bar.get_x() + bar.get_width()/2, h + 0.3,
                    str(int(h)), ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

print(f"Seahawks — AM: {sea_am}, FM: {sea_fm}")
print(f"Patriots — AM: {pat_am}, FM: {pat_fm}")

---
## Step 5 — States With the Most Affiliates
Let's see which states have the highest concentration of affiliates for each team.
The `.value_counts()` method is one of the most useful tools in Pandas — it counts
how many times each unique value appears.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Seahawks — top 10 states
sea_states = seahawks['state'].value_counts().head(10)
sea_states.plot(kind='barh', ax=axes[0], color='#69BE28', edgecolor='black')
axes[0].set_title('Seahawks: Top 10 States')
axes[0].set_xlabel('Number of Stations')
axes[0].invert_yaxis()

# Patriots — top 10 states
pat_states = patriots['state'].value_counts().head(10)
pat_states.plot(kind='barh', ax=axes[1], color='#C60C30', edgecolor='black')
axes[1].set_title('Patriots: Top 10 States')
axes[1].set_xlabel('Number of Stations')
axes[1].invert_yaxis()

plt.tight_layout()
plt.show()

---
## Step 6 — Power (ERP) Comparison
ERP = Effective Radiated Power. Let's compare the power profiles of each team's network.
A **box plot** shows the median, quartiles, and outliers — great for seeing the "shape"
of a distribution at a glance.

In [ ]:
# Tag each DataFrame so we can combine them
seahawks_copy = seahawks.copy()
patriots_copy = patriots.copy()
seahawks_copy['team'] = 'Seahawks'
patriots_copy['team'] = 'Patriots'
combined = pd.concat([seahawks_copy, patriots_copy], ignore_index=True)

# Convert ERP to numeric (some may be blank)
combined['erp'] = pd.to_numeric(combined['erp'], errors='coerce')

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# AM power comparison
am_data = combined[combined['band'] == 'AM']
if len(am_data) > 0:
    am_data.boxplot(column='erp', by='team', ax=axes[0],
                    patch_artist=True,
                    boxprops=dict(facecolor='#FFD700', edgecolor='black'))
    axes[0].set_title('AM Station Power (Watts)')
    axes[0].set_ylabel('Power (Watts)')
    axes[0].set_xlabel('')

# FM ERP comparison
fm_data = combined[combined['band'] == 'FM']
if len(fm_data) > 0:
    fm_data.boxplot(column='erp', by='team', ax=axes[1],
                    patch_artist=True,
                    boxprops=dict(facecolor='#1E90FF', edgecolor='black'))
    axes[1].set_title('FM Station ERP (kW)')
    axes[1].set_ylabel('ERP (kW)')
    axes[1].set_xlabel('')

plt.suptitle('')  # Remove auto-title from boxplot
plt.tight_layout()
plt.show()

# Print summary stats
print("=== AM Power Summary (Watts) ===")
if len(am_data) > 0:
    print(am_data.groupby('team')['erp'].describe()[['count','mean','min','max']].to_string())
print()
print("=== FM ERP Summary (kW) ===")
if len(fm_data) > 0:
    print(fm_data.groupby('team')['erp'].describe()[['count','mean','min','max']].to_string())

---
## Step 7 — Frequency Distribution (AM)
AM stations cluster on certain frequencies (especially clear-channel 50 kW stations).
A **histogram** shows how the frequencies are distributed across the AM band.

In [ ]:
sea_am_freqs = seahawks[seahawks['band'] == 'AM']['frequency'].astype(float)
pat_am_freqs = patriots[patriots['band'] == 'AM']['frequency'].astype(float)

fig, ax = plt.subplots(figsize=(10, 4))

bins = range(530, 1710, 10)  # AM band: 530-1700 kHz in 10 kHz steps

if len(sea_am_freqs) > 0:
    ax.hist(sea_am_freqs, bins=bins, alpha=0.6, label='Seahawks', color='#69BE28', edgecolor='black')
if len(pat_am_freqs) > 0:
    ax.hist(pat_am_freqs, bins=bins, alpha=0.6, label='Patriots', color='#C60C30', edgecolor='black')

ax.set_xlabel('Frequency (kHz)')
ax.set_ylabel('Number of Stations')
ax.set_title('AM Frequency Distribution')
ax.legend()
plt.tight_layout()
plt.show()

---
## Step 8 — Interactive Map With Folium
**Folium** renders Leaflet.js maps right inside the notebook. Each station gets a
color-coded marker (green = Seahawks, red = Patriots). Click a marker to see callsign,
frequency, and band.

Folium comes pre-installed on Google Colab — no setup needed!

In [ ]:
import folium

# Center the map on the continental US
m = folium.Map(location=[39.5, -98.35], zoom_start=4, tiles='CartoDB positron')

def add_markers(df, team_name, color, icon_name):
    """Add a circle marker for each station in the DataFrame."""
    for _, row in df.iterrows():
        if pd.notna(row['lat']) and pd.notna(row['lon']):
            popup_text = (
                f"<b>{row['callsign']}</b><br>"
                f"{row['frequency']} {row['band']}<br>"
                f"{row.get('city', '')} {row.get('state', '')}<br>"
                f"ERP: {row.get('erp', 'N/A')}"
            )
            folium.CircleMarker(
                location=[row['lat'], -abs(row['lon'])],  # lon stored as positive in some records
                radius=5,
                color=color,
                fill=True,
                fill_color=color,
                fill_opacity=0.7,
                popup=folium.Popup(popup_text, max_width=200),
                tooltip=f"{row['callsign']} ({row['band']})"
            ).add_to(m)

add_markers(seahawks, 'Seahawks', '#69BE28', 'signal')
add_markers(patriots, 'Patriots', '#C60C30', 'signal')

# Simple legend
legend_html = '''
<div style="position: fixed; bottom: 30px; left: 30px; z-index: 1000;
            background: white; padding: 10px; border: 2px solid grey;
            border-radius: 5px; font-size: 14px;">
  <b>NFL Radio Affiliates</b><br>
  <i style="color:#69BE28;">&#11044;</i> Seahawks<br>
  <i style="color:#C60C30;">&#11044;</i> Patriots
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

m

---
## Step 9 — Geographic Reach: How Far Do the Affiliates Spread?
We can measure the geographic spread of each team's network by computing the
distance from each affiliate to the team's home stadium using the **Haversine formula**
(the standard way to measure distance on a sphere).

In [ ]:
from math import radians, sin, cos, sqrt, atan2

def haversine(lat1, lon1, lat2, lon2):
    """Distance in miles between two lat/lon points."""
    R = 3958.8  # Earth radius in miles
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    return R * 2 * atan2(sqrt(a), sqrt(1 - a))

# Stadium coordinates
STADIUMS = {
    'Seahawks': (47.5952, -122.3316),  # Lumen Field, Seattle
    'Patriots': (42.0909, -71.2643),   # Gillette Stadium, Foxborough
}

def add_distance(df, team_key):
    """Add a 'distance_mi' column measuring miles from each station to the stadium."""
    slat, slon = STADIUMS[team_key]
    distances = []
    for _, row in df.iterrows():
        if pd.notna(row['lat']) and pd.notna(row['lon']):
            d = haversine(row['lat'], -abs(row['lon']), slat, slon)
            distances.append(d)
        else:
            distances.append(None)
    df = df.copy()
    df['distance_mi'] = distances
    return df

seahawks_d = add_distance(seahawks, 'Seahawks')
patriots_d = add_distance(patriots, 'Patriots')

# Plot the distance distributions
fig, ax = plt.subplots(figsize=(10, 4))

sea_dist = seahawks_d['distance_mi'].dropna()
pat_dist = patriots_d['distance_mi'].dropna()

ax.hist(sea_dist, bins=20, alpha=0.6, label='Seahawks', color='#69BE28', edgecolor='black')
ax.hist(pat_dist, bins=20, alpha=0.6, label='Patriots', color='#C60C30', edgecolor='black')
ax.set_xlabel('Distance from Stadium (miles)')
ax.set_ylabel('Number of Stations')
ax.set_title('How Far Do Affiliates Reach From the Home Stadium?')
ax.legend()
plt.tight_layout()
plt.show()

print(f"Seahawks — avg distance: {sea_dist.mean():.0f} mi, farthest: {sea_dist.max():.0f} mi")
print(f"Patriots — avg distance: {pat_dist.mean():.0f} mi, farthest: {pat_dist.max():.0f} mi")

# Show farthest affiliates
print("\n--- Farthest Seahawks Affiliate ---")
if len(seahawks_d.dropna(subset=['distance_mi'])) > 0:
    far_sea = seahawks_d.loc[seahawks_d['distance_mi'].idxmax()]
    print(f"{far_sea['callsign']} {far_sea['frequency']} {far_sea['band']} — {far_sea.get('city','')} {far_sea.get('state','')} ({far_sea['distance_mi']:.0f} mi)")

print("\n--- Farthest Patriots Affiliate ---")
if len(patriots_d.dropna(subset=['distance_mi'])) > 0:
    far_pat = patriots_d.loc[patriots_d['distance_mi'].idxmax()]
    print(f"{far_pat['callsign']} {far_pat['frequency']} {far_pat['band']} — {far_pat.get('city','')} {far_pat.get('state','')} ({far_pat['distance_mi']:.0f} mi)")

---
## Step 10 — Summary Scorecard
Let's pull everything together into one comparison table so we can see the
head-to-head matchup at a glance.

In [ ]:
combined['erp'] = pd.to_numeric(combined['erp'], errors='coerce')

scorecard = pd.DataFrame({
    'Metric': [
        'Total Affiliates',
        'AM Stations',
        'FM Stations',
        'AM %',
        'States Covered',
        'Avg Distance from Stadium (mi)',
        'Farthest Affiliate (mi)',
        'Avg AM Power (W)',
        'Avg FM ERP (kW)',
    ],
    'Seahawks': [
        len(seahawks),
        sea_am,
        sea_fm,
        f"{sea_am / max(len(seahawks),1) * 100:.0f}%",
        seahawks['state'].nunique(),
        f"{sea_dist.mean():.0f}" if len(sea_dist) else 'N/A',
        f"{sea_dist.max():.0f}" if len(sea_dist) else 'N/A',
        f"{am_data[am_data['team']=='Seahawks']['erp'].mean():.0f}" if len(am_data[am_data['team']=='Seahawks']) else 'N/A',
        f"{fm_data[fm_data['team']=='Seahawks']['erp'].mean():.1f}" if len(fm_data[fm_data['team']=='Seahawks']) else 'N/A',
    ],
    'Patriots': [
        len(patriots),
        pat_am,
        pat_fm,
        f"{pat_am / max(len(patriots),1) * 100:.0f}%",
        patriots['state'].nunique(),
        f"{pat_dist.mean():.0f}" if len(pat_dist) else 'N/A',
        f"{pat_dist.max():.0f}" if len(pat_dist) else 'N/A',
        f"{am_data[am_data['team']=='Patriots']['erp'].mean():.0f}" if len(am_data[am_data['team']=='Patriots']) else 'N/A',
        f"{fm_data[fm_data['team']=='Patriots']['erp'].mean():.1f}" if len(fm_data[fm_data['team']=='Patriots']) else 'N/A',
    ],
})

# Display as a nicely formatted table
scorecard_styled = scorecard.style.set_properties(**{
    'text-align': 'center',
    'font-size': '13px'
}).set_properties(
    subset=['Metric'], **{'text-align': 'left', 'font-weight': 'bold'}
).hide(axis='index')

display(scorecard_styled)

# Determine winner in each category
print("\n=== Quick Takeaways ===")
if len(seahawks) > len(patriots):
    print(f"📻 Seahawks have more affiliates ({len(seahawks)} vs {len(patriots)})")
elif len(patriots) > len(seahawks):
    print(f"📻 Patriots have more affiliates ({len(patriots)} vs {len(seahawks)})")
else:
    print(f"📻 Both teams are tied at {len(seahawks)} affiliates!")

print(f"📡 Seahawks: {sea_am} AM / {sea_fm} FM  |  Patriots: {pat_am} AM / {pat_fm} FM")
print(f"🗺️  Seahawks cover {seahawks['state'].nunique()} states  |  Patriots cover {patriots['state'].nunique()} states")

---
### That's a Wrap!

**What we covered:**
1. Uploading & loading CSV data with Pandas
2. Counting and comparing with `.value_counts()` and `len()`
3. Bar charts, grouped bars, histograms, and box plots with Matplotlib
4. Interactive mapping with Folium (Leaflet.js under the hood)
5. Haversine distance calculations
6. Building a summary scorecard

All of this data comes from the **Radioland** broadcast database — the same
system that powers the interactive coverage maps at [radioland.net](https://radioland.net).

**Want to try your own analysis?** Swap in any two NFL teams, or try comparing
MLB, NBA, or NHL affiliates — the database has them all.